In [ ]:
import os
import glob
import zipfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# extract the params from all .sqx file xml and add it to db
def getStrategiesParams(df, df3, df4, hourEnter, hourExit, path):
    for file in glob.glob(path + "*.sqx"):
        zf = zipfile.ZipFile(file, 'r')
        for name in zf.namelist():
            if name == "strategy_Portfolio.xml":
                f = zf.open(name)
                x = f.read().decode("utf-8")
                # extract parameter from selected line found in xml
                entry_line = str([line for line in x.split('\n') if '<Param key="#IntraWeekHour#" controlType="jspinnerVar"' in line])
                exit_line = str([line for line in x.split('\n') if '<Param key="#Number#" controlType="jspinner"' in line])
                # Get parameter
                entry = int(entry_line.split('>')[1].split('<')[0])
                exit = int(exit_line.split('>')[1].split('<')[0])
                
                df3.at[entry,'Count'] += 1
                
                exitHour = addParamtoArray(df, entry, exit)
                hourEnter.append(entry)
                hourExit.append(exitHour)
                #df1.at[entry,'HourEnter'] = df1.iloc[entry]['HourEnter'] + 1
                #df1.at[exitHour,'HourExit'] = df1.iloc[exitHour]['HourExit'] + 1
                #df1.at[entry,'HourEnter'] = df1.iloc[entry]['HourEnter'] + 1
                #df2.at[exitHour,'HourExit'] = df2.iloc[exitHour]['HourExit'] + 1
                #print(str(entry) + " " + str(exit) + " " + str(exitHour))
                
                try:
                    df4.at[exitHour,'Count'] += 1
                except:
                    print("PAIN: " + exitHour)
                
# add a parameter to the db and return exit hour         
def addParamtoArray(df, entry, exit):
    count = 0
    #loop from entry to exit
    while (count < exit):
        if entry == 115:
            entry = 0
        #add 1 entry to db
        df.at[entry,'Count'] = df.iloc[entry]['Count'] + 1
        count += 1
        entry += 1
    # set exit hour from looped entry
    exitHour = entry
    return exitHour
    
# main
def main():
    for subdir, dirs, files in os.walk("../data/SQX Seasonality Histogram/"):
        for dirc in dirs:
            df = pd.DataFrame({'Count': [0] * 115})
            hourEnter = []
            hourExit = []
            #df = pd.DataFrame({"Hour": np.arange(115, 'Count': [0] * 115)
            
            df1 = pd.DataFrame({'HourEnter': [0] * 115})
            df2 = pd.DataFrame({'HourExit': [0] * 115})
            df3 = pd.DataFrame({'Count': [0] * 115})
            df4 = pd.DataFrame({'Count': [0] * 120})
            
            getStrategiesParams(df, df3, df4, hourEnter, hourExit, os.path.join(subdir, dirc) + "/")
            
            # print db
            plt.bar(df.index, df['Count'])
            plt.title('IntraWeek Hours Histogram - ' + dirc)
            plt.xlabel('Hour of Week')
            plt.ylabel('Count')
            plt.show()
            
            
            # Initialise the subplot function using number of rows and columns
            plt.bar(df3.index, df3['Count'], alpha=0.3, color='green')
            plt.bar(df4.index, df4['Count'], alpha=0.3, color='red')
            plt.title('IntraWeek Hours Histogram - ' + dirc)
            plt.xlabel('Exit Hour of Week')
            plt.ylabel('Count')
            plt.show()
            #second set
            plt.bar(df3.index, df3['Count'], alpha=0.75, color='green')
            plt.title('IntraWeek Enter Hours Histogram - ' + dirc)
            plt.xlabel('Entry Hour of Week')
            plt.ylabel('Count')
            plt.show()
            plt.bar(df4.index, df4['Count'], alpha=0.75, color='red')
            plt.title('IntraWeek Exit Hours Histogram - ' + dirc)
            plt.xlabel('Exit Hour of Week')
            plt.ylabel('Count')
            plt.show()
            
            
            
            
            
            plt.scatter(hourEnter, hourExit, color = 'green')
            plt.title('IntraWeek Hours Scatterplot - ' + dirc)
            plt.xlabel('Entry Hour')
            plt.ylabel('Exit Hour')
            plt.show()


main()